In [1]:
import tkinter as tk
from tkinter import scrolledtext, messagebox
from keras.models import load_model
import json
import pickle
import numpy as np
import nltk
import random

model = load_model('chatbot_model.h5')

intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

lemmatizer = nltk.stem.WordNetLemmatizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words if word.lower() not in stop_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

def send():
    msg = entry.get()
    if msg.strip() != "":
        response = chatbot_response(msg)
        txt.config(state=tk.NORMAL)
        txt.insert(tk.END, "You: " + msg + "\n\n")
        txt.insert(tk.END, "Bot: " + response + "\n\n")
        txt.config(state=tk.DISABLED)
        entry.delete(0, tk.END)
    else:
        messagebox.showerror("Error", "Please enter a message.")

# Create GUI window
root = tk.Tk()
root.title("Chatbot GUI")

# Create Chat History Window
txt = scrolledtext.ScrolledText(root, width=60, height=20)
txt.grid(row=0, column=0, columnspan=2, padx=10, pady=10)
txt.config(state=tk.DISABLED)

# Create Entry Box
entry = tk.Entry(root, width=50)
entry.grid(row=1, column=0, padx=10, pady=10)

# Create Send Button
send_btn = tk.Button(root, text="Send", width=10, command=send)
send_btn.grid(row=1, column=1, padx=10, pady=10)

# Run the GUI
root.mainloop()



1/1 [==============================] - 0s 160ms/step


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\gicha\anaconda3\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\gicha\AppData\Local\Temp\ipykernel_9096\4125762573.py", line 63, in send
    response = chatbot_response(msg)
  File "C:\Users\gicha\AppData\Local\Temp\ipykernel_9096\4125762573.py", line 57, in chatbot_response
    res = getResponse(ints, intents)
  File "C:\Users\gicha\AppData\Local\Temp\ipykernel_9096\4125762573.py", line 47, in getResponse
    tag = ints[0]['intent']
IndexError: list index out of range


1/1 [==============================] - 0s 31ms/step
